In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

In [2]:
data = pd.read_csv("gene_x_train.csv", low_memory=False)

In [3]:
data2=data.drop(columns=["Unnamed: 0"])

In [5]:
data3=data2.drop(columns=["gene"])

In [6]:
data3.head()

,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A3GALT2,A4GALT,A4GNT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,6.59,6.74,5.44,13.02,5.81,12.69,4.21,0.00,8.49,3.33,...,10.64,10.38,7.55,8.62,11.02,6.23,10.51,11.67,11.91,11.06
1,5.59,5.13,4.13,13.75,7.15,14.45,3.11,1.80,10.47,1.14,...,8.79,8.75,8.99,9.05,11.07,6.92,11.46,10.00,12.05,11.87
2,6.21,6.16,1.76,13.92,7.22,15.26,1.76,0.72,10.45,0.00,...,8.90,8.84,8.92,9.22,11.18,7.56,11.47,9.81,12.12,11.64
3,4.89,5.76,5.23,14.51,6.97,14.18,2.02,0.00,9.17,4.72,...,10.20,10.34,8.36,8.86,11.00,6.88,11.67,10.45,12.06,11.74
4,6.01,6.46,4.12,13.43,6.61,16.44,3.00,0.00,9.02,2.79,...,10.04,9.81,7.97,8.68,10.93,7.25,11.59,9.99,12.35,11.55


In [7]:
data_y = pd.read_csv("gene_y_train.csv", low_memory=False)

In [8]:
arr_y=data_y.values
y=arr_y[:,1]

In [9]:
data_test = pd.read_csv("gene_x_test.csv", low_memory=False)

In [10]:
data_test2=data_test.drop(columns=["Unnamed: 0"])
data_test3=data_test2.drop(columns=["gene"])

In [11]:
from sklearn.preprocessing import StandardScaler

In [12]:
input_dim = 36514
encoding_dim = 15

In [13]:
input_layer = Input(shape=(input_dim,))

In [14]:
# Encoder layers
encoded = Dense(2048, activation='relu')(input_layer)
encoded = Dense(1024, activation='relu')(encoded)
encoded = Dense(512, activation='relu')(encoded)
encoded = Dense(256, activation='relu')(encoded)
encoded = Dense(128, activation='relu')(encoded)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)

# Decoder layers
decoded = Dense(32, activation='relu')(encoded)
decoded = Dense(64, activation='relu')(decoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(256, activation='relu')(decoded)
decoded = Dense(512, activation='relu')(decoded)
decoded = Dense(1024, activation='relu')(decoded)
decoded = Dense(2048, activation='relu')(decoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

# autoencoder model
autoencoder = Model(input_layer, decoded)

# encoder model
encoder = Model(input_layer, encoded)

# compiling the autoencoder
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# standard scaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(data3)
x_test_scaled = scaler.transform(data_test3)

# training the autoencoder
autoencoder.fit(x_train_scaled, x_train_scaled, epochs=50, batch_size=256, shuffle=True, validation_data=(x_test_scaled, x_test_scaled))

# encoding the data
x_train_encoded = encoder.predict(x_train_scaled)
x_test_encoded = encoder.predict(x_test_scaled)

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - loss: 0.6931 - val_loss: 0.6924
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.6925 - val_loss: 0.6890
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.6891 - val_loss: 0.6219
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.6252 - val_loss: 0.0143
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.0481 - val_loss: -2.9435
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: -1.9873 - val_loss: -29.3340
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: -21.8375 - val_loss: -136.5383
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: -104.6204 - val_loss: -539.3335
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: -416.5614 - val_loss: -1899.6343
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: -1474.1865 - val_loss: -5892.6558
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: -4576.5952 - val_loss: -16841.1172
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: -13082.336

In [15]:
x_train_encoded.shape

(129, 15)

In [16]:
x_test_encoded.shape

(33, 15)

In [17]:
x_train_encoded[5]

array([ 0.        ,  0.        ,  0.        ,  0.98167056,  0.        ,
        0.        ,  0.        , 33.373703  ,  0.        ,  0.        ,
        0.        ,  0.        , 36.983246  ,  0.        ,  0.        ],
      dtype=float32)

In [29]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [100,200,500,1000, 1500]
# Number of features to consider at every split
max_features = ['sqrt']
# Maximum number of levels in tree
max_depth = [5,10,15,20,30,40,50,70]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'class_weight':['balanced']}

# Use the random grid to search for best hyperparameters
# First create the base model to tune
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random_encoded = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
rf_random_encoded.fit(x_train_encoded, y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'class_weight': ['balanced'],
                                        'max_depth': [5, 10, 15, 20, 30, 40, 50,
                                                      70, None],
                                        'max_features': ['sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 500, 1000,
                                                         1500]},
                   random_state=42, verbose=2)

In [30]:
rf_random_encoded.best_params_

{'n_estimators': 500,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 40,
 'class_weight': 'balanced',
 'bootstrap': True}

In [31]:
rf_random_encoded.best_score_

0.9844961240310077

In [32]:
best_random_encoded = rf_random_encoded.best_estimator_

In [33]:
y_pred_encoded=best_random_encoded.predict(x_test_encoded)

In [34]:
y_pred_encoded

array(['C', 'C', 'C', 'C', 'C', 'NC', 'C', 'NC', 'C', 'NC', 'C', 'C', 'C',
       'C', 'C', 'C', 'C', 'NC', 'C', 'NC', 'C', 'NC', 'NC', 'NC', 'NC',
       'C', 'C', 'C', 'NC', 'NC', 'C', 'C', 'C'], dtype=object)

In [35]:
y_test = pd.read_csv("gene_y_test.csv", low_memory=False)

In [36]:
arr_y_test=y_test.values
arr_y_test[:,1]

array(['C', 'C', 'C', 'C', 'C', 'NC', 'C', 'NC', 'C', 'NC', 'C', 'C', 'C',
       'C', 'C', 'C', 'C', 'NC', 'C', 'NC', 'C', 'NC', 'NC', 'NC', 'NC',
       'C', 'C', 'C', 'NC', 'NC', 'C', 'C', 'C'], dtype=object)

In [37]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [38]:
accuracy_score(arr_y_test[:,1], y_pred_encoded)

1.0

In [39]:
cm = confusion_matrix(arr_y_test[:,1], y_pred_encoded)
print(cm)

[[22  0]
 [ 0 11]]
